# UAS Flight Analysis: CloudySky

## Setup

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import sys

from datetime import datetime, timedelta

import hvplot
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import xarray as xr

import holoviews as hv
from bokeh.models.renderers import GlyphRenderer
from bokeh.models import Range1d, LinearAxis, DatetimeTickFormatter

hv.extension('bokeh')

# sys.path.insert(0, "/home/derek/Software/python/envDataSystem_analysis/utilities")
flight_path = os.path.abspath("")
proj_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(""))))
code_path = os.path.join(proj_path, "code")
if proj_path not in sys.path:
    sys.path.insert(0, proj_path)
if code_path not in sys.path:
    sys.path.insert(0, code_path)
# print(sys.path)
from convert import Data, dt_to_string, string_to_dt
from dataevent import (
    DataEvent,
    save_for_invert_msems,
    load_inverted_msems,
    process_msems,
    process_cdp,
    save_flight_itx
)
import config.settings

### Data systems
Define the datasystem(s). This includes base paths, instruments with variable maps and extra parameters specific to each instrument. This should change to much flight to flight but the paths might need to be changed to point to the data.

In [1]:
flight_datasystems = {
    "CloudySky": {
        "uas_cloudy": {
            # "base_path": "/home/derek/Data/envDataSystem/from_cloudbase/UIServer/cloudy1.acg.pmel.noaa.gov/cloudysky",
            "base_path": config.settings.ENVDSYS_CLOUDYSKY_DATA_PATH,
            "instruments": {
                "ambient_trh": {
                    "format": "envdsys",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("temperature", "ambient_T", "ambient_temperature"),
                        (
                            "relative_humidity",
                            "ambient_rh",
                            "ambient_relative_humidity",
                        ),
                    ],
                },
                "fast_trh": {
                    "format": "envdsys",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("temperature", "fast_ambient_T", "fast_ambient_temperature"),
                        (
                            "relative_humidity",
                            "fast_ambient_rh",
                            "fast_ambient_relative_humidity",
                        ),
                    ],
                },
                "msems_sheath_trh": {
                    "format": "envdsys",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("temperature", "msems_sh_T", "msems_sheath_temperature"),
                        (
                            "relative_humidity",
                            "msems_sh_rh",
                            "msems_sheath_relative_humidity",
                        ),
                    ],
                },
                "cloudy_cdp": {
                    "format": "envdsys",
                    "timebase": 1,
                    "dims": ["time", "cdp_bins"],
                    "process_options": {"type": "cdp"},
                    "variables": [
                        ("bin_counts", "cdp_bin_counts", "cloudy_cdp_bin_counts"),
                        ("diameter_um", "cdp_dp_um_2d", "cloudy_cdp_diameter_um_2d"),
                        ("integral_counts", "cdp_intN", "cloudy_cdp_integral_counts"),
                        (
                            "laser_current",
                            "cdp_laser_current",
                            "cloudy_cdp_laser_current",
                        ),
                        # 'dump_spot_monitor',
                        (
                            "wingboard_temperature",
                            "cdp_wingboard_T",
                            "cloudy_cdp_wingboard_temperature",
                        ),
                        (
                            "laser_temperature",
                            "cdp_laser_T",
                            "cloudy_cdp_laser_temperature",
                        ),
                        (
                            "control_board_temperature",
                            "cdp_control_board_T",
                            "cloudy_cdp_control_board_temperature",
                        ),
                        # ('average_transit',
                    ],
                },
                "cloudy_msems": {
                    "format": "envdsys",
                    "timebase": 30,
                    "resample": False,
                    "dims": ["time", "msems_bins"],
                    "invert_path": os.path.join(flight_path, "data/msems_inversion"),
                    "process_options": {"type": "msems"},
                    "variables": [
                        ("bin_counts", "msems_bin_counts", "cloudy_msems_bin_counts"),
                        ("diameter_um", "msems_dp_um_2d", "cloudy_msems_diameter_um"),
                        (
                            "scan_direction",
                            "msems_scan_direction",
                            "cloudy_msems_scan_direction",
                        ),
                        (
                            "sheath_flow_avg",
                            "msems_sheath_flow_avg",
                            "cloudy_msems_sheath_flow_avg",
                        ),
                        (
                            "sheath_flow_sd",
                            "msems_sheath_flow_sd",
                            "cloudy_msems_sheath_flow_sd",
                        ),
                        (
                            "sample_flow_avg",
                            "msems_sample_flow_avg",
                            "cloudy_msems_sample_flow_avg",
                        ),
                        (
                            "sample_flow_sd",
                            "msems_sample_flow_sd",
                            "cloudy_msems_sample_flow_sd",
                        ),
                        (
                            "pressure_avg",
                            "msems_pressure_avg",
                            "cloudy_msems_pressure_avg",
                        ),
                        (
                            "pressure_sd",
                            "msems_pressure_sd",
                            "cloudy_msems_pressure_sd",
                        ),
                        (
                            "temperature_avg",
                            "msems_T_avg",
                            "cloudy_msems_temperature_avg",
                        ),
                        ("temperature_sd", "msems_T_sd", "cloudy_msems_temprature_sd"),
                        # 'msems_error',
                        (
                            "mcpc_sample_flow",
                            "mcpc_sample_flow",
                            "cloudy_msems_mcpc_sample_flow",
                        ),
                        (
                            "mcpc_saturator_flow",
                            "mcpc_saturator_flow",
                            "cloudy_msems_mcpc_saturator_flow",
                        ),
                        (
                            "mcpc_condenser_temp",
                            "mcpc_condenser_T",
                            "cloudy_msems_mcpc_condenser_temp",
                        ),
                        # 'mcpc_error',
                        ("bin_time", "msems_bin_time", "cloudy_msems_bin_time"),
                        ("scan_type", "msems_scan_type", "cloudy_msems_scan_type"),
                        (
                            "plumbing_time",
                            "msems_plumbing_time",
                            "cloudy_msems_plumbing_time",
                        ),
                    ],
                },
            },
        }
    },
    "GroundStation": {
        "uasground": {
            # "base_path": "/home/derek/Data/envDataSystem/from_cloudbase",
            "base_path": config.settings.ENVDSYS_GROUDSTATION_DATA_PATH,
            # "base_path": "/home/derek/Data/envDataSystem/from_cloudbase/UIServer/cloudy1.acg.pmel.noaa.gov/UASGroundTest",
            "instruments": {
                "amcpc_cn": {
                    "format": "envdsys",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("concentration", "ground_cn", "amcpc_cn_concentration"),
                    ],
                },
                # "magic_cn": {
                #     "format": "envdsys",
                #     "timebase": 1,
                #     "dims": ["time"],
                #     "variables": [
                #         ("concentration", "magci_cn", "magic_cn_concentration"),
                #     ]
                # }
            },
        }
    },
    "AutoPilot": {
        "navigation": {
            # "base_path": "/home/derek/Data/UAS/autopilot/piccolo", 
            "base_path": "./data/piccolo",
            "instruments": {
                "piccolo": {
                    "format": "piccolo-log",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("Lat", "latitude", ""),
                        ("Lon", "longitude", ""),
                        ("Height", "altitude", ""),
                        ("GroundSpeed", "ground_speed", ""),
                        ("Direction", "heading", ""),
                        ("BaroAlt", "pressure_altitude", ""),
                        ("TAS", "true_air_speed", ""),
                        ("Roll", "roll", ""),
                        ("Pitch", "pitch", ""),
                        ("Yaw", "yaw", ""),
                        ("MagHdg", "heading_mag", ""),
                        ("AGL", "height_agl", ""),
                    ]
                }
            }
        }
    }
}

NameError: name 'config' is not defined

## Flight Details

### Flight ID

In [ ]:
project = "<ProjectID>"
project_long_name = "<Long Project Name>"
platform = "<PlatformName>" # FVR-55, AeroPhys, RHBrown
flight_id = "<Flight_??>"

### Event Data

In [ ]:
# shouldn't change this if using the templates
payload_id = "CloudySky"

flight_event_config = {
    "preflight": {
            "start_time": "2022-02-01T21:00:00Z",
            "end_time": "2022-02-01T22:00:00Z",
            "datasystems": ["CloudySky", "GroundStation"],
        },
    "flight": {
        "start_time": "2022-02-16T20:00:00Z",
        "end_time": "2022-02-16T22:15:00Z",
        "datasystems": ["CloudySky", "GroundStation"],
    },
    "postflight": {},
}

# --- automatically generated - don't edit below this line ---
flight_config = {
    "kind": "CloudySkyFlight",
    "metadata": {"project": project, "platform": platform, "flight_id": flight_id, "payload_id": payload_id},
    "events": flight_event_config,
    "datasystems": flight_datasystems,
}
# flight_config

## Load Data
Un/comment lines as needed. Multiple events can be processed if wanted. Will load data based on Event Data cell above

### Create Event(s)

In [ ]:
# create preflight event
# preflight_event = DataEvent("preflight", config=flight_config)

# create flight event
flight_event = DataEvent("flight", config=flight_config)

# create postflight event
# postflight_event = DataEvent("postflight", config=flight_config)

### Invert mSEMS data 
Save msems data and invert using Igor. Simply process each file that is listed in the output from the previous cell. You don't need to save any data, the inversion function automatically writes a file to same folder as the input file. The save/load cycle needs to be done separately for each event type

#### Pre-flight

In [ ]:
# saves files for use in Igor
save_for_invert_msems(
    "cloudy_msems",
    event_id="preflight",
    datasystem="CloudySky",
    controller="uas_cloudy",
    config=flight_config,
)

In [ ]:
# reads files from Igor inversion
load_inverted_msems(
    "cloudy_msems",
    event_id="preflight",
    datasystem="CloudySky",
    controller="uas_cloudy",
    config=flight_config,
    data=preflight_event.get_data(),
)

#### Flight

In [ ]:
save_for_invert_msems(
    "cloudy_msems",
    event_id="flight",
    datasystem="CloudySky",
    controller="uas_cloudy",
    config=flight_config,
)

In [ ]:
load_inverted_msems(
    "cloudy_msems",
    event_id="flight",
    datasystem="CloudySky",
    controller="uas_cloudy",
    config=flight_config,
    data=flight_event.get_data(),
)

#### Post-flight

In [ ]:
save_for_invert_msems(
    "cloudy_msems",
    event_id="postflight",
    datasystem="CloudySky",
    controller="uas_cloudy",
    config=flight_config,
)

In [ ]:
load_inverted_msems(
    "cloudy_msems",
    event_id="postflight",
    datasystem="CloudySky",
    controller="uas_cloudy",
    config=flight_config,
    data=preflight_event.get_data(),
)

### Process data

#### mSEMS

In [ ]:
# pre_msems = preflight_event.get_dataset("cloudy_msems", datasystem="CloudySky", controller="uas_cloudy")
# process_msems(pre_msems)

flight_event.process_msems(
    "cloudy_msems", datasystem="CloudySky", controller="uas_cloudy"
)
msems = flight_event.get_dataset(
    "cloudy_msems", datasystem="CloudySky", controller="uas_cloudy"
)
# process_msems(msems)

# post_msems = flight_event.get_dataset("cloudy_msems", datasystem="CloudySky", controller="uas_cloudy")
# process_msems(post_msems)

#### CDP

In [ ]:
# pre_msems = preflight_event.get_dataset("cloudy_cdp", datasystem="CloudySky", controller="uas_cloudy")
# process_msems(pre_msems)

flight_event.process_cdp("cloudy_cdp", datasystem="CloudySky", controller="uas_cloudy")
cdp = flight_event.get_dataset(
    "cloudy_cdp", datasystem="CloudySky", controller="uas_cloudy"
)
# process_cdp(cdp)

# post_msems = flight_event.get_dataset("cloudy_cdp", datasystem="CloudySky", controller="uas_cloudy")
# process_msems(post_msems)

#### Autopilot

In [10]:
flight_event.process_piccolo("piccolo", datasystem="AutoPilot", controller="navigation")
nav = flight_event.get_dataset("piccolo", datasystem="AutoPilot", controller="navigation")
# process_cdp(cdp)


### Merge datasets
Combine datasets on a common timebase. User can specify timebase (tb=1 is default) and list of datasets (ds_list). If ds_list is omitted, all available datasets are merged.

In [ ]:
# preflight = preflight_event.merge()
# preflight_30s = preflight_event.merge(tb=30, ds_list=["cloudy_msems", "fast_trh"])

flight = flight_event.merge()
flight_30s = flight_event.merge(tb=30, ds_list=["cloudy_msems", "piccolo", "ambient_rh", "fast_trh", "amcpc_cn"])

# postflight = postflight_event.merge()
# postflight_30s = postflight_event.merge(tb=30, ds_list=["cloudy_msems", "fast_trh"])

### Save datasets

In [ ]:
# save_flight_itx(preflight)
# save_flight_itx(preflight_30s)

save_flight_itx(flight)
save_flight_itx(flight_30s)

# save_flight_itx(postflight)
# save_flight_itx(postflight_30s)

## Data Analysis
Add descriptive text in [markdown](https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html) cells and run code in the default code cells. 

*This would be description of particular data analysis*

In [2]:
# code cell
flight

NameError: name 'flight' is not defined

In [ ]:
flight_30s

### Visualization

#### Standard Plot Definitions
This is a list of standard plots that we might want to see. Run the following cell to create the plots for use below. 

In [ ]:
dt_formatter = DatetimeTickFormatter(
    years="%Y", months="%Y-%m", days="%F", hours="%m-%d %H:%M", hourmin="%m-%d %H:%M", minutes="%H:%M", minsec="%T", seconds="%T", milliseconds="%T.%1N"
)

msems_dndlogdp_im = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"time": "time_mid", "msems_bins": "msems_dp_um"})
    .reset_coords()
    .msems_dNdlogDp.hvplot.quadmesh(
        x="time_mid",
        y="msems_dp_um",
        logy=True,
        cmap="rainbow",
        responsive=True,
        min_width=300,
        min_height=300,
        xformatter=dt_formatter,
        label="mSEMS dNdlogDp"
    )
)
msems_dndlogdp_scans = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"msems_bins": "msems_dp_um"})
    .reset_coords()
    .msems_dNdlogDp.hvplot.line(
        x="msems_dp_um", logx=True, responsive=True, min_width=300, min_height=300, label="mSEMS dNdlogDp"
    )
)

msems_dsdlogdp_im = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"time": "time_mid", "msems_bins": "msems_dp_um"})
    .reset_coords()
    .msems_dSdlogDp.hvplot.quadmesh(
        x="time_mid",
        y="msems_dp_um",
        logy=True,
        cmap="rainbow",
        responsive=True,
        min_width=300,
        min_height=300,
        xformatter=dt_formatter,
        label="mSEMS dSdlogDp"
    )
)
msems_dsdlogdp_scans = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"msems_bins": "msems_dp_um"})
    .reset_coords()
    .msems_dSdlogDp.hvplot.line(
        x="msems_dp_um", logx=True, responsive=True, min_width=300, min_height=300, label="mSEMS dSdlogDp"
    )
)

msems_dvdlogdp_im = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"time": "time_mid", "msems_bins": "msems_dp_um"})
    .reset_coords()
    .msems_dVdlogDp.hvplot.quadmesh(
        x="time_mid",
        y="msems_dp_um",
        logy=True,
        cmap="rainbow",
        responsive=True,
        min_width=300,
        min_height=300,
        xformatter=dt_formatter,
        label="mSEMS dVdlogDp"
    )
)
msems_dvdlogdp_scans = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"msems_bins": "msems_dp_um"})
    .reset_coords()
    .msems_dVdlogDp.hvplot.line(
        x="msems_dp_um", logx=True, responsive=True, min_width=300, min_height=300, label="mSEMS dVdlogDp"
    )
)

msems_intN_ts = flight_30s.msems_intN.hvplot.scatter(
    label="mSEMS intN", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)
msems_intS_ts = flight_30s.msems_intS.hvplot.scatter(
    label="mSEMS intS", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)
msems_intV_ts = flight_30s.msems_intV.hvplot.scatter(
    label="mSEMS intV", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)
cdp_lwc_ts = flight.cdp_lwc.hvplot.scatter(
    label="LWC", c="blue", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)

if "altitude" in flight_30s:
    msems_intN_alt = (
        flight_30s.set_coords("msems_intN")
        .swap_dims({"time": "msems_intN"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="msems_intN",
            y="altitude",
            label="mSEMS intN",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )
    msems_intS_alt = (
        flight_30s.set_coords("msems_intN")
        .swap_dims({"time": "msems_intS"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="msems_intS",
            y="altitude",
            label="mSEMS intS",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )
    msems_intV_alt = (
        flight_30s.set_coords("msems_intN")
        .swap_dims({"time": "msems_intV"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="msems_intV",
            y="altitude",
            label="msems intV",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )
    cdp_lwc_alt = (
        flight.set_coords("cdp_lwc")
        .swap_dims({"time": "cdp_lwc"})
        .reset_coords()[["altitude", "ambient_rh"]]
        .hvplot.scatter(
            x="cdp_lwc",
            y="altitude",
            c="ambient_rh",
            label="LWC",
            cmap="rainbow",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )

if "ground_cn" in flight:
    ground_cn_ts = flight.ground_cn.hvplot.scatter(
        label="ground cn", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
    )

#### Data Plots
Compose plots by using the assigned name above (or create new ones). Use "+" and "\*" to create plots side-by-side or overlayed, respectively. E.g., 

```msems_intN + msems_intS``` 

will display plots of intN and intS side by side and will share the time axis.

In [ ]:
# Example compositions
# msems_dndlogdp_im + msems_dndlogdp_scans
# msems_dndlogdp_im + msems_dsdlogdp_im + msems_dvdlogdp_im
# msems_intN_ts + msems_intS_ts + msems_intV_ts
